##  Optimization manual steps

In [1]:
import random
import numpy as np
import pprint
from mycrograd_debug.engine_debug import Value
from mycrograd_debug.nn_debug import MLP
from mycrograd_debug.drawviz_debug import (
    draw_dot,
    draw_nn,
)
from mycrograd_debug.util_debug import debugFunc

np.random.seed(1337)
random.seed(1337)
number_of_iterations = 8
pp = pprint.PrettyPrinter(indent=4)
global activation
global loss
global step
step=0

makeimg = True
debug_steps = True

#### Multi Layer Perceptron 1-2-1

In [2]:
# initialize a model
nin = 1  # number of inputs
nout = 1  # number of outputs
Value.value_counter = 0
model = MLP(nin, [2, nout], weightsinit=2, lastReLU=False, debug_bw=True)
xinumbers = list(range(4, 4 + nin))
xinput = [Value(x, type="i%s" % index) for index, x in enumerate(xinumbers, start=1)]
# xtarget = Value(1.2, type="t")  # desired targets
xtarget = Value(0.0, type="t")  # desired targets
debugFunc(model, {"parameters"}, message="start", inputs=xinput, targets=xtarget)

Module nn MLP: structure [1, 2, 1]
start
parameters
 name lay neu ty   data   grad
 v001  L1  N1 w1   0.50   0.00
 v002  L1  N1  b   0.00   0.00
 v003  L1  N2 w1   0.50   0.00
 v004  L1  N2  b   0.00   0.00
 v005  L2  N1 w1   0.60   0.00
 v006  L2  N1 w2   1.20   0.00
 v007  L2  N1  b   0.00   0.00
inputs
[Value(name=v008,layernumber=,neuronnumber=,weightnumber=,type=i1,data=4, grad=0)]
targets
Value(name=v009,layernumber=,neuronnumber=,weightnumber=,type=t,data=0.0, grad=0)


In [3]:
# loss function single MLP
def loss_single(activation, target):
    total_loss = (activation - target)*(activation - target)
    total_loss.type="l"
    return total_loss

In [4]:
def imageFunc(filename):
    # print(filename)
    if makeimg:
        dot1=draw_dot(activation)
        dot1.render("images/"+filename , format="svg", view=True)
        dot2=draw_nn(xinput, model)
        dot2.node(
            name="loss", label="step %2d loss %6.2f" % (step, loss.data), shape="record"
        )
        dot2.render("images/nn/"+filename, format="svg", view=True)

In [5]:
def getactivation(filename="default"):
    #### forward pass0
    global model
    global loss
    global activation
    global step
    step = step + 1
    activation = model(xinput)
    loss = loss_single(activation, xtarget)
    # debugFunc(model, {"parameters"}, message="act")
    debugFunc(model, {}, message="act")
    imageFunc("step%d_1loss" % step)


def zeroGradients(filename="default"):
    global model
    global step
    model.zero_grad()
    for i in xinput:
        i.grad = 0
    # print("zero'd gradients")
    # debugFunc(model, {"parameters"}, message="zer")
    debugFunc(model, {}, message="zer")
    imageFunc("step%d_2zero" % step)


def backward(filename="default"):
    #### backward pass
    global activation
    global step
    activation.backward()
    # print("parameters after backpass")
    # debugFunc(model, {"parameters"}, message="bwd")
    debugFunc(model, {}, message="bwd")
    imageFunc("step%d_3back" % step)


def updateParams(filename="default"):
    #### update
    global model
    global step
    for p in model.parameters():
        p.data += -0.05 * p.grad
    # print("updated parameters")
    # debugFunc(model, {"parameters"}, message="upd")
    debugFunc(model, {}, message="upd")
    imageFunc("step%d_3upda" % step)


In [6]:
def optStep(filename="default"):
    global model
    global step
    getactivation()
    zeroGradients()
    backward()
    updateParams()
    print(f"step %3d output %6.4f loss %6.4f" % (step, activation.data, loss.data))
    dot = draw_nn(xinput, model)
    dot.render("static/" + filename)


In [7]:
def opt():
    global loss

    for k in range(number_of_iterations):
        print("start step %d" % k)
        optStep(filename="default")
        # print(f"step %3d output %6.4f loss %6.4f" % (k, activation.data, loss.data))

opt()

start step 0
act
zer
backward add v015            0.00 ->   1.00  v016            0.00 ->   1.00
backward mul v006 L2 N1 w2   0.00 ->   2.00  v013 L1 N2  a   0.00 ->   1.20
backward add v004 L1 N2  b   0.00 ->   1.20  v012            0.00 ->   1.20
backward mul v003 L1 N2 w1   0.00 ->   4.80  v008       i1   0.00 ->   0.60
backward add v007 L2 N1  b   0.00 ->   1.00  v014            0.00 ->   1.00
backward mul v005 L2 N1 w1   0.00 ->   2.00  v011 L1 N1  a   0.00 ->   0.60
backward add v002 L1 N1  b   0.00 ->   0.60  v010            0.00 ->   0.60
backward mul v001 L1 N1 w1   0.00 ->   2.40  v008       i1   0.60 ->   0.90
bwd
upd


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from message bus without replying
/snap/core20/curren

step   1 output 3.6000 loss 12.9600
start step 1
act
zer
backward add v030            0.00 ->   1.00  v031            0.00 ->   1.00
backward mul v006 L2 N1 w2   0.00 ->   0.98  v028 L1 N2  a   0.00 ->   1.10
backward add v004 L1 N2  b   0.00 ->   1.10  v027            0.00 ->   1.10
backward mul v003 L1 N2 w1   0.00 ->   4.40  v008       i1   0.00 ->   0.29
backward add v007 L2 N1  b   0.00 ->   1.00  v029            0.00 ->   1.00
backward mul v005 L2 N1 w1   0.00 ->   1.49  v026 L1 N1  a   0.00 ->   0.50
backward add v002 L1 N1  b   0.00 ->   0.50  v025            0.00 ->   0.50
backward mul v001 L1 N1 w1   0.00 ->   2.00  v008       i1   0.29 ->   0.48
bwd
upd


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from message bus without replying
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from message bus without replying
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_in

step   2 output 1.7730 loss 3.1435
start step 2
act
zer
backward add v045            0.00 ->   1.00  v046            0.00 ->   1.00
backward add v007 L2 N1  b   0.00 ->   1.00  v044            0.00 ->   1.00
backward mul v005 L2 N1 w1   0.00 ->   1.06  v041 L1 N1  a   0.00 ->   0.43
backward add v002 L1 N1  b   0.00 ->   0.43  v040            0.00 ->   0.43
backward mul v001 L1 N1 w1   0.00 ->   1.70  v008       i1   0.00 ->   0.12
backward mul v006 L2 N1 w2   0.00 ->   0.04  v043 L1 N2  a   0.00 ->   1.05
backward add v004 L1 N2  b   0.00 ->   1.05  v042            0.00 ->   1.05
backward mul v003 L1 N2 w1   0.00 ->   4.20  v008       i1   0.12 ->   0.16
bwd
upd


...

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from message bus without replying
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from message bus without replying
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-li

upd
step   7 output -5.1647 loss 26.6737
start step 7
act
zer
backward add v120            0.00 ->   1.00  v121            0.00 ->   1.00
backward mul v006 L2 N1 w2   0.00 ->  -4.78  v118 L1 N2  a   0.00 ->   1.49
backward add v004 L1 N2  b   0.00 ->   1.49  v117            0.00 ->   1.49
backward mul v003 L1 N2 w1   0.00 ->   5.98  v008       i1   0.00 ->  -1.64
backward add v007 L2 N1  b   0.00 ->   1.00  v119            0.00 ->   1.00
backward mul v005 L2 N1 w1   0.00 ->  -0.44  v116 L1 N1  a   0.00 ->   0.32
backward add v002 L1 N1  b   0.00 ->   0.32  v115            0.00 ->   0.32
backward mul v001 L1 N1 w1   0.00 ->   1.28  v008       i1  -1.64 ->  -1.66
bwd


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from message bus without replying
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from mes

upd
step   8 output -7.6410 loss 58.3851


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from message bus without replying
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/tmeng12/snap/code/common/.cache/gio-modules/libgiolibproxy.so


In [8]:
# optStep(filename="default")

In [9]:
# optStep(filename="default")

In [10]:
# optStep(filename="default")
# optStep(filename="default")

eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from message bus without replying
